In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)

In [4]:
from MatrixFactorization.NMFRecommender import NMFRecommender

In [5]:
recommender= NMFRecommender(URM_all)

NMFRecommender: URM Detected 1079 (4.15 %) cold items.


In [6]:
recommender.fit(num_factors=258, solver="multiplicative_update", init_type="nndsvda", beta_loss="frobenius")

NMFRecommender: Computing NMF decomposition...


/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:700: RuntimeWarning: overflow encountered in true_divide
  numerator /= denominator
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:813: RuntimeWarning: invalid value encountered in multiply
  H *= delta_H
/home/infamouskz/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("MF_NMF_recommendations.csv")